# Sentiment Classification


### Generate Word Embeddings and retrieve outputs of each layer with Keras based on Classification task

Word embeddings are a type of word representation that allows words with similar meaning to have a similar representation.

It is a distributed representation for text that is perhaps one of the key breakthroughs for the impressive performance of deep learning methods on challenging natural language processing problems.

We willl use the imdb dataset to learn word embeddings as we train our dataset. This dataset contains 25,000 movie reviews from IMDB, labeled with sentiment (positive or negative). 



### Dataset

`from keras.datasets import imdb`

Dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers). For convenience, the words are indexed by their frequency in the dataset, meaning the for that has index 1 is the most frequent word. Use the first 20 words from each review to speed up training, using a max vocab size of 10,000.

As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.


### Aim

1. Build a Sequential Model using Keras for Sentiment Classification task. 
2. Report the Accuracy of the model. 
3. Retrive the output of each layer in keras for a given single test sample from the trained model you built.


#### Usage:

In [ ]:
from keras.datasets import imdb

vocab_size = 10000 #vocab size

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size) # vocab_size is no.of words to consider from the dataset, ordering based on frequency.

In [ ]:
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
vocab_size = 10000 #vocab size
maxlen = 300  #number of word used from each review
#load dataset as a list of ints
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
#make all sequences of the same length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test =  pad_sequences(x_test, maxlen=maxlen)

Using TensorFlow backend.


17465344/17464789 [==============================] - 1s 0us/step


In [ ]:
x_train.shape #number of review, number of words in each review

(25000, 300)

To take a look at the review and sentiment:

In [ ]:
(training_data, training_labels), (test_data, test_labels)= imdb.load_data(num_words=vocab_size, index_from=3)

In [ ]:
word_to_id = imdb.get_word_index()
word_to_id = {k:(v+3) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2

1646592/1641221 [==============================] - 0s 0us/step


In [ ]:
id_to_word = {value:key for key,value in word_to_id.items()}
print(' '.join(id_to_word[id] for id in training_data[6] ))
print('The sentiment is:', training_labels[6])

<START> lavish production values and solid performances in this straightforward adaption of jane <UNK> satirical classic about the marriage game within and between the classes in <UNK> 18th century england northam and paltrow are a <UNK> mixture as friends who must pass through <UNK> and lies to discover that they love each other good humor is a <UNK> virtue which goes a long way towards explaining the <UNK> of the aged source material which has been toned down a bit in its harsh <UNK> i liked the look of the film and how shots were set up and i thought it didn't rely too much on <UNK> of head shots like most other films of the 80s and 90s do very good results
The sentiment is: 1


Here is a postive review number 6 from the training set. 

## Build Keras Embedding Layer Model
We can think of the Embedding layer as a dicionary that maps a index assigned to a word to a word vector. This layer is very flexible and can be used in a few ways:

* The embedding layer can be used at the start of a larger deep learning model. 
* Also we could load pre-train word embeddings into the embedding layer when we create our model.
* Use the embedding layer to train our own word2vec models.

The keras embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unqiue intger number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

In [ ]:
import time
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding
from keras.layers import LSTM, TimeDistributed
model = Sequential()
model.add(Embedding(10000, 100, input_length=maxlen)) #10000 for vocab size, 8 for dimensionality of embedding
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(100)))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 300, 100)          1000000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 300, 64)           42240     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 300, 100)          6500      
_________________________________________________________________
flatten_1 (Flatten)          (None, 30000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 250)               7500250   
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 251       
Total params: 8,549,241
Trainable params: 8,549,241
Non-trainable params: 0
_________________________________________________________________


In [ ]:
start = time.clock()
history = model.fit(x_train, y_train, epochs=5, batch_size=128, validation_data=(x_test, y_test))
end = time.clock()
print('Time spent:', end-start)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 173s 7ms/step - loss: 0.3865 - acc: 0.8096 - val_loss: 0.3057 - val_acc: 0.8692
Epoch 2/5
25000/25000 [==============================] - 169s 7ms/step - loss: 0.1861 - acc: 0.9285 - val_loss: 0.2952 - val_acc: 0.8773
Epoch 3/5
25000/25000 [==============================] - 172s 7ms/step - loss: 0.0936 - acc: 0.9677 - val_loss: 0.3896 - val_acc: 0.8636
Epoch 4/5
25000/25000 [==============================] - 171s 7ms/step - loss: 0.0339 - acc: 0.9886 - val_loss: 0.5582 - val_acc: 0.8580
Epoch 5/5
25000/25000 [==============================] - 170s 7ms/step - loss: 0.0143 - acc: 0.9954 - val_loss: 0.8053 - val_acc: 0.8598
Time spent: 1262.687027


In [ ]:
score = model.evaluate(x_test, y_test)

25000/25000 [==============================] - 184s 7ms/step


In [ ]:
score

[0.8053167874073982, 0.85976]

In [ ]:
print('\nTest Acc: %.2f%%' %(score[1]*100))


Test Acc: 85.98%


In [ ]:
import numpy as np
model.predict(np.array([x_test[11],]))

array([[0.00787853]], dtype=float32)

In [ ]:
model.layers[0].output

<tf.Tensor 'embedding_3/embedding_lookup/Identity:0' shape=(?, 300, 100) dtype=float32>

In [ ]:
model.layers[4].output

<tf.Tensor 'dense_3/Relu:0' shape=(?, 250) dtype=float32>

In [ ]:
model.layers[4].output

<tf.Tensor 'dense_3/Relu:0' shape=(?, 250) dtype=float32>

In [ ]:
from keras import backend as K

inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
functors = [K.function([inp, K.learning_phase()], [out]) for out in outputs]    # evaluation functions

# Testing
test = np.array([x_test[11],])
layer_outs = [func([test, 1.]) for func in functors]
print(len(layer_outs[0][0][0][0]))

100
